# 7.0 Model Explainability & Deep Inference Logging
**Project:** VibeCheck AI  
**Status:** Post-Deployment Audit Phase

## Overview
In the previous notebooks (4.0 and 5.0), we trained a **Multinomial Naive Bayes** model and built a basic inference pipeline. However, for a production-grade system, "knowing the label" isn't enough. 

This notebook focuses on **XAI (Explainable AI)**. We will extract the internal statistical weights of the model to log exactly *why* a decision was made. This is crucial for debugging our limited dataset and verifying the impact of our emoji-amplification logic.

### Key Objectives:
1. Load production artifacts (Model & Vectorizer).
2. Extract **Feature Log Probabilities** to identify decision drivers.
3. Implement a **Deep Logging Function** that flags high-ambiguity predictions.
4. Visualize the **Global Feature Importance** of our current vocabulary.

In [4]:
import joblib
import pandas as pd
import numpy as np
from emoji_sentiment_analysis.config import MODELS_DIR
from emoji_sentiment_analysis.features import extract_emojis

# Load the production artifacts verified in 4.0 and 5.0
model = joblib.load(MODELS_DIR / "sentiment_model.pkl")
vectorizer = joblib.load(MODELS_DIR / "tfidf_vectorizer.pkl")

print("✅ Artifacts loaded. Ready for deep logging.")

✅ Artifacts loaded. Ready for deep logging.


## 1. The Decision Audit Engine
The `generate_inference_log` function below transforms raw text into a rich data object. Unlike standard inference, this function:
* **Calculates Feature Weights:** Subtracts the Negative log-probability from the Positive log-probability for every token in the input.
* **Flags High Ambiguity:** Uses an **Entropy Threshold** (0.15) to identify cases where the model is "guessing" rather than "knowing."
* **Tracks Engine Input:** Captures the exact string (including amplified emojis) that the classifier processed.

In [5]:
def generate_inference_log(text):
    # 1. Pipeline Execution
    emojis = extract_emojis(text)
    engine_input = f"{text} {emojis * 5}"
    vec = vectorizer.transform([engine_input])
    
    # 2. Probability Extraction (From 5.0 logic)
    probs = model.predict_proba(vec)[0]
    prediction = int(model.predict(vec)[0])
    
    # 3. Feature Importance Extraction
    # feature_log_prob_ contains: [log_prob_neg, log_prob_pos]
    feature_names = vectorizer.get_feature_names_out()
    nonzero_indices = vec.nonzero()[1]
    
    feature_impacts = []
    for idx in nonzero_indices:
        token = feature_names[idx]
        # Weight = LogProb(Pos) - LogProb(Neg). Positive favors Pos class.
        weight = model.feature_log_prob_[1][idx] - model.feature_log_prob_[0][idx]
        feature_impacts.append({
            "token": token,
            "weight": round(weight, 4),
            "sentiment_lean": "Positive" if weight > 0 else "Negative"
        })
    
    # 4. Construct the JSON-like log
    log_entry = {
        "raw_text": text,
        "engine_input": engine_input,
        "prediction": "Positive" if prediction == 1 else "Negative",
        "confidence": round(float(np.max(probs)), 4),
        "entropy_flag": "High Ambiguity" if abs(probs[0] - probs[1]) < 0.15 else "Clear Signal",
        "top_drivers": sorted(feature_impacts, key=lambda x: abs(x['weight']), reverse=True)[:3],
        "emoji_detected": len(emojis) > 0
    }
    
    return log_entry

# Test the logger
test_log = generate_inference_log("I love this! 😊")
import json
print(json.dumps(test_log, indent=2))

{
  "raw_text": "I love this! \ud83d\ude0a",
  "engine_input": "I love this! \ud83d\ude0a \ud83d\ude0a\ud83d\ude0a\ud83d\ude0a\ud83d\ude0a\ud83d\ude0a",
  "prediction": "Positive",
  "confidence": 0.5723,
  "entropy_flag": "High Ambiguity",
  "top_drivers": [
    {
      "token": "love",
      "weight": 0.3929,
      "sentiment_lean": "Positive"
    },
    {
      "token": "this",
      "weight": -0.0658,
      "sentiment_lean": "Negative"
    }
  ],
  "emoji_detected": true
}


## 2. Global Model Signal Analysis
To understand the "personality" of our model, we can look at the **Feature Importance** across the entire dataset. 

By calculating the difference in log-probabilities across the whole vocabulary, we identify the specific tokens that the model considers its "strongest evidence." This allows us to ensure that our emoji-weighting (w=5) is having the intended effect relative to standard vocabulary words like "happy" or "thanks."

In [6]:
# Map entire vocabulary
vocab = vectorizer.get_feature_names_out()
weights = model.feature_log_prob_[1] - model.feature_log_prob_[0]

importance_df = pd.DataFrame({'token': vocab, 'weight': weights})

print("🔥 Top 10 POSITIVE Signals in Model:")
display(importance_df.sort_values('weight', ascending=False).head(10))

print("\n❄️ Top 10 NEGATIVE Signals in Model:")
display(importance_df.sort_values('weight', ascending=True).head(10))

🔥 Top 10 POSITIVE Signals in Model:


,token,weight
48,happy,3.269052
47,great,1.664616
107,thanks,1.561104
105,thank,1.480214
106,thank you,1.480214
101,smile,1.440196
108,thanks for,1.431364
84,our,1.421771
85,out,1.379906
40,for the,1.334843



❄️ Top 10 NEGATIVE Signals in Model:


,token,weight
133,unhappy,-3.352139
96,sad,-1.967292
90,please,-1.331317
73,miss,-1.269606
14,because,-1.264431
0,10,-1.190984
54,him,-1.185950
102,so,-1.150225
65,koalas,-1.117758
104,still,-1.096806


## 📊 Summary of Findings
* **Interpretability:** We can now provide a "Reasoning" list for every prediction (e.g., "😊 had a weight of +2.1").
* **Reliability:** The `entropy_flag` allows the system to warn users when a prediction is statistically weak (Confidence ~50-60%).

## 🚀 Next Steps: Production Integration
The logic developed in this notebook can now be ported to the production environment:
1. **`predict.py`**: Update the core prediction script to include the `log_entry` generation.
2. **`main.py`**: Save these logs to a `data/logs/inference_history.csv` for future retraining.
3. **UI Update**: Display the `top_drivers` in the frontend so users understand the AI's "thought process."